<a href="https://colab.research.google.com/github/LukasFeldler/SemAI23/blob/main/UE03_LuFe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#UE03 - SPARQL Query

Please complete the 10 tasks in the `2. SPARQL` sheet of `SemAI.jar` first, and then transfer the task descriptions and your solutiosn in executable form to this notebook.

## Preparation

Reuse imports and functions from https://github.com/jku-win-dke/SemAI/blob/main/V02_SPARQL.ipynb and load the solar system graph. Query the solar system graph to check that everything works fine. 

In [2]:
# Install required packages
!pip install -q rdflib

# Imports
import pandas as pd
from rdflib import Graph, Literal, RDF, URIRef, BNode, Namespace
from rdflib.namespace import FOAF , XSD , RDFS, NamespaceManager 

# Convenient Functions
def sparql_select(graph,query,use_prefixes=True):
  results = graph.query(query)          # execute the query against the graph, resulting in a rdflib.plugins.sparql.processor.SPARQLResult
  rows = []                             # a list of dictionaries, as intermediate format to construct the pandas DataFrame
  for result in results:                # iterate over the result set of the query, a result is an instance of rdflib.query.ResultRow
    row = {}                            #     create a dictionary to hold a single row of the result
    for var in results.vars:            #     iterate over the variables of the SPARQLResult to add a dictionary entry for each variable
      if (isinstance(result[var],URIRef) and use_prefixes):
        row[var] = result[var].n3(graph.namespace_manager)   # use namespace prefixes to shorten URIs
      else:
        row[var] = result[var]                  
    rows.append(row)                    #     add the dictionary (row) to the list 
  return pd.DataFrame(rows,columns=results.vars)        
                                        # return a pandas DataFrame constructed from the list of dictionaries, with the variables from the result set as columns      

def sparql_construct(graph, query):
  result_graph = Graph(namespace_manager = g.namespace_manager)  # create a Graph object that reuses the namespace prefixes of the original graph
  result_graph += graph.query(query)                             # execute the construct query against the original graph and add the resulting graph to the new one
  return result_graph

def sparql_ask(graph, query):
  return bool(graph.query(query))      # an ASK query has a boolean result, which should be returned as such

# Load solar system graph
g = Graph()
g.parse("https://raw.githubusercontent.com/jku-win-dke/SemAI/main/data/solarsystem.ttl",format="turtle")

# Query solar system graph (to check that everything works fine)
df = sparql_select(g,"""
  SELECT ?planet ?apoapsis ?apoapsis_uom
  WHERE { 
    ?planet rdf:type dbo:Planet . 
    OPTIONAL { ?planet v:apoapsis [rdf:value ?apoapsis ; v:uom ?apoapsis_uom ].  }
  }
""")
df

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 KB 4.9 MB/s eta 0:00:00


,planet,apoapsis,apoapsis_uom
0,:Mercury,0.467,unit:AU
1,:Venus,0.728,unit:AU
2,:Earth,1.017,unit:AU
3,:Earth,149597871,unit:KM
4,:Mars,1.666,unit:AU
5,:Jupiter,5.4588,unit:AU
6,:Saturn,9.0412,unit:AU
7,:Uranus,20.11,unit:AU
8,:Neptune,30.33,unit:AU



## Task 1 (1 pt)

*TODO: add task description*

In [4]:
task1 = sparql_select(g,"""SELECT ?d ?match
WHERE {
  ?d rdf:type dbo:DwarfPlanet .
  OPTIONAL { ?d skos:exactMatch ?match . }
}
ORDER BY ?d""")
task1

,d,match
0,:Ceres,dbr:1_Ceres
1,:Eris,None
2,:Haumea,None
3,:Makemake,None
4,:Pluto,dbr:Pluto


## Task 2 (1 pt)

*TODO: add task description*

In [5]:
task2 = sparql_select(g,"""SELECT ?star ?match ?massVal ?massUoM
WHERE {
  ?star rdf:type dbo:Star .
  ?star skos:exactMatch ?match . 
  ?star v:mass ?mass .
  ?mass v:uom ?massUoM ;
        rdf:value ?massVal .
           
}
ORDER BY ?star""")
task2

,star,match,massVal,massUoM
0,:AlphaCentauriA,wd:Q2090157,1.1,v:SolarMass
1,:AlphaCentauriB,wd:Q1052548,0.9,v:SolarMass
2,:ProximaCentauri,wd:Q14266,0.1221,v:SolarMass
3,:Sun,dbr:Sun,1.9884e+30,unit:KG


## Task 3 (1 pt)

*TODO: add task description*

In [6]:
task3 = sparql_select(g,"""Select ?planet ?assertedNo (count(?moon) as ?noOfDescribedMoons)
where{
?planet rdf:type dbo:Planet.
?planet v:nrOfMoons ?assertedNo.
optional{
?moon rdf:type dbo:Satellite.
?moon v:orbits ?planet.
}
}
group by ?planet ?assertedNo
order by ?planet
""")
task3

,planet,assertedNo,noOfDescribedMoons
0,:Earth,1,1
1,:Jupiter,79,4
2,:Mars,2,2
3,:Mercury,0,0
4,:Neptune,14,0
5,:Saturn,82,2
6,:Uranus,27,0
7,:Venus,0,0


## Task 4 (1 pt)

*TODO: add task description*

In [7]:
task4 = sparql_select(g,"""select ((?countMoons/?countPlanets) as ?avgNoOfDescribedMoons)
where {
   {
      select (count(?moons) as ?countMoons)
      where {
         ?moons rdf:type dbo:Satellite.
      }
   }

   {
      select (count(?planets) as ?countPlanets)
      where {
         ?planets rdf:type dbo:Planet.
      }
   }
}
""")
task4

,avgNoOfDescribedMoons
0,1.125


## Task 5 (1 pt)

*TODO: add task description*

In [8]:
task5 = sparql_select(g,"""SELECT ?class (COUNT(?instance) as ?noOfInstances)
WHERE {
  ?instance rdf:type ?class.
}
GROUP BY ?class
HAVING (COUNT(?instance) > 1)
ORDER BY ?class
""")
task5

,class,noOfInstances
0,dbo:DwarfPlanet,5
1,dbo:Planet,8
2,dbo:Satellite,9
3,dbo:Star,4


## Task 6 (1 pt)

*TODO: add task description*

In [9]:
task6 = sparql_select(g,"""SELECT ?class ?prop (COUNT(DISTINCT ?instance) as ?noOfInstances)
WHERE {
  ?instance rdf:type ?class .
  ?instance ?prop ?value . 
}
GROUP BY ?class ?prop
HAVING (COUNT(DISTINCT ?instance) >= 7)
ORDER BY ?class ?prop
""")
task6

,class,prop,noOfInstances
0,dbo:Planet,v:apoapsis,8
1,dbo:Planet,v:nrOfMoons,8
2,dbo:Planet,v:orbits,8
3,dbo:Planet,v:radius,7
4,dbo:Planet,v:temperature,7
5,dbo:Planet,rdf:type,8
6,dbo:Planet,skos:exactMatch,8
7,dbo:Satellite,v:orbits,9
8,dbo:Satellite,v:radius,9
9,dbo:Satellite,rdf:type,9


## Task 7 (1 pt)

*TODO: add task description*

In [10]:
task7 = sparql_select(g,"""SELECT DISTINCT ?x ?y
WHERE {
  ?x rdf:type ?type.
  FILTER (?type = dbo:Star || ?type = dbo:Satellite)
  OPTIONAL {
    ?x v:orbits ?y.
    ?y rdf:type dbo:Planet
  }
}
ORDER BY ?x
""")
task7

,x,y
0,:AlphaCentauriA,None
1,:AlphaCentauriB,None
2,:Callisto,:Jupiter
3,:Deimos,:Mars
4,:Enceladus,:Saturn
5,:Europa,:Jupiter
6,:Ganymede,:Jupiter
7,:Io,:Jupiter
8,:Mimas,:Saturn
9,:Moon,:Earth


## Task 8 (1 pt)

*TODO: add task description*

In [ ]:
# TODO: include and execute the query, display the result

## Task 9 (1 pt)

*TODO: add task description*

In [11]:
task9 = sparql_ask(g,"""ASK{
  ?star a dbo:Star .
  ?start v:mass ?massStar.
  ?massStar rdf:value ?massStarVal.
  ?sun a :Sun.
  ?sun v:mass ?massSun.
  ?massSun rdf:value ?massSunVal;
  Filter(?massStarVal > ?massSunVal)
}""")
task9

False

## Task 10 (1 pt)

*TODO: add task description*

In [ ]:
# TODO: include and execute the query, display the result